# Funcionalidades básicas

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('linear_regression').getOrCreate()

23/12/26 18:46:54 WARN Utils: Your hostname, Valhalla-Zorin resolves to a loopback address: 127.0.1.1; using 192.168.100.36 instead (on interface enp4s0)
23/12/26 18:46:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/26 18:46:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
training = spark.read.format('libsvm').load('Data/sample_linear_regression_data.txt')

23/12/26 18:48:43 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [5]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [6]:
 lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')

In [7]:
lrModel = lr.fit(training)

23/12/26 18:53:11 WARN Instrumentation: [99dda99f] regParam is zero, which might cause numerical instability and overfitting.
23/12/26 18:53:11 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/12/26 18:53:11 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
23/12/26 18:53:11 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [8]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [9]:
lrModel.intercept

0.14228558260358093

In [10]:
training_summary = lrModel.summary

In [11]:
training_summary.r2

0.027839179518600154

In [12]:
training_summary.rootMeanSquaredError

10.16309157133015

# Lendo e separando os dados em treino/teste

In [13]:
data = spark.read.format('libsvm').load('Data/sample_linear_regression_data.txt')

23/12/26 18:55:56 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [14]:
train_data, test_data = data.randomSplit([0.7,0.3])

In [20]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                336|
|   mean|0.40522043985230416|
| stddev| 10.716572544993632|
|    min|-28.571478869743427|
|    max| 27.111027963108548|
+-------+-------------------+



In [21]:
test_data.describe().show()

+-------+--------------------+
|summary|               label|
+-------+--------------------+
|  count|                 165|
|   mean|-0.04516829012535...|
| stddev|    9.47843114290769|
|    min| -26.805483428483072|
|    max|   27.78383192005107|
+-------+--------------------+



In [22]:
lr_model = lr.fit(train_data)

23/12/26 18:58:26 WARN Instrumentation: [766ad8c0] regParam is zero, which might cause numerical instability and overfitting.


In [24]:
results = lr_model.evaluate(test_data)

In [25]:
results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-28.423788890492503|
| -28.30511517165511|
| -19.19118724108135|
|-16.260683562039876|
|-17.269045558802528|
|-19.175074498864046|
| -19.14389043571905|
|-17.083610166215713|
|-16.319961082598514|
|-14.822658861754949|
| -19.78508311198186|
|-16.123579473557154|
|-15.063222294664996|
|-18.083341353448642|
|-14.579077565520379|
|-12.235922937923236|
|-10.717114518409996|
|-14.749988825096546|
|-12.677558940433688|
|-15.664453527059393|
+-------------------+
only showing top 20 rows



In [26]:
results.rootMeanSquaredError

9.594102427323328

In [30]:
results.predictions.show()

+-------------------+--------------------+--------------------+
|              label|            features|          prediction|
+-------------------+--------------------+--------------------+
|-26.805483428483072|(10,[0,1,2,3,4,5,...|  1.6183054620094302|
|-22.949825936196074|(10,[0,1,2,3,4,5,...|   5.355289235459036|
|-22.837460416919342|(10,[0,1,2,3,4,5,...| -3.6462731758379885|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|  -2.907609060923885|
|-17.803626188664516|(10,[0,1,2,3,4,5,...| -0.5345806298619868|
|-17.428674570939506|(10,[0,1,2,3,4,5,...|  1.7463999279245406|
|-17.065399625876015|(10,[0,1,2,3,4,5,...|  2.0784908098430366|
|-17.026492264209548|(10,[0,1,2,3,4,5,...|0.057117902006163956|
| -16.71909683360509|(10,[0,1,2,3,4,5,...|-0.39913575100657517|
| -16.08565904102149|(10,[0,1,2,3,4,5,...|  -1.263000179266542|
|-15.780685032623301|(10,[0,1,2,3,4,5,...|   4.004398079358557|
|-15.732088272239245|(10,[0,1,2,3,4,5,...|  0.3914912013179067|
|-15.375857723312297|(10,[0,1,2,3,4,5,..

In [31]:
unlabeled_data = test_data.select('features')

In [32]:
predictions = lr_model.transform(unlabeled_data)

In [33]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...|  1.6183054620094302|
|(10,[0,1,2,3,4,5,...|   5.355289235459036|
|(10,[0,1,2,3,4,5,...| -3.6462731758379885|
|(10,[0,1,2,3,4,5,...|  -2.907609060923885|
|(10,[0,1,2,3,4,5,...| -0.5345806298619868|
|(10,[0,1,2,3,4,5,...|  1.7463999279245406|
|(10,[0,1,2,3,4,5,...|  2.0784908098430366|
|(10,[0,1,2,3,4,5,...|0.057117902006163956|
|(10,[0,1,2,3,4,5,...|-0.39913575100657517|
|(10,[0,1,2,3,4,5,...|  -1.263000179266542|
|(10,[0,1,2,3,4,5,...|   4.004398079358557|
|(10,[0,1,2,3,4,5,...|  0.3914912013179067|
|(10,[0,1,2,3,4,5,...|-0.31263542864730076|
|(10,[0,1,2,3,4,5,...|   2.723796473615965|
|(10,[0,1,2,3,4,5,...|  1.5391495014157637|
|(10,[0,1,2,3,4,5,...| -0.7419257874688666|
|(10,[0,1,2,3,4,5,...| -2.0561124808412004|
|(10,[0,1,2,3,4,5,...|   2.249215039741491|
|(10,[0,1,2,3,4,5,...|    0.19827872898219|
|(10,[0,1,2,3,4,5,...|   3.83738